# Python Tutorial III: Modules and Classes

In today's tutorial we'll learn how to build our own python modules and classes.  We'll start with modules:

## Modules

### When to use:
* Your script gets very long and you want to have easier maintenance. 
* You want to reuse a function in several programs or scripts without copy/paste.
* Performance reasons.

### What is it?
* A file containing Python definitions and statements.
* The file name is the module name with the suffix .py appended.
* Within a module, the module's name is available as the valuable of the global variable *__name__*.

Let's look at an example:

In [ ]:
# module difference.py
def for_diff(function,x=0,h=.1):
    deriv=(function(x+h)-function(x))/h
    return deriv

def back_diff(function,x=0,h=.1):
    deriv=(function(x)-function(x-h))/h
    return deriv

def cent_diff(function,x=0,h=.1):
    deriv=(function(x+h)-function(x-h))/(2*h)
    return derive

We **import** modules just like we import libraries like ``numpy``, ``math``, and ``scipy``

In [ ]:
%matplotlib inline   
# differences.py should be located within the same directory as this lecture and is included in the repo
import differences as diff 
import matplotlib.pyplot as plt
import numpy as np

def myfun(x=0):
    import numpy as np
    return np.exp(-x**2)*np.sin(np.pi*x)

def myder(x=0):
    from math import sin,exp,pi
    temp=np.exp(-x**2)
    return temp*(pi*np.cos(np.pi*x)-2*x*np.sin(np.pi*x))
    
X=np.linspace(0,1,100)
dX=diff.for_diff(myfun,X,.01)
abs_err=np.abs(dX-myder(X))

plt.plot(X,abs_err)
plt.show

## Where does the module file need to go?

Say you are trying to **import spam**.

When imported, the interpreter searches for spam in(in order):
1. A built-in module with that name. 
2. *spam.py* in a list of directories given by the variable *sys.path*. 
    1. The directory containing the input script (or the current directory when no file is specified).
    2. PYTHONPATH (a list of directory names, syntax as shell variable PATH).
    3. The installation-dependent default.

## Generalizations of Modules: Packages

Packages are modules with a directory structure.
You can even make packages with subpackages and simply exploit the dot.dot reference to navigate through the package to get to the function you want (e.g. matplotlib.pyplot.plot).  
If you want to develop a well-comparmentalized package you can look at online help: https://python-packaging.readthedocs.io/en/latest/


 

## Classes

Classes are a basic feature of object-oriented programming.  They are a little more complex than the *struct* features in Matlab.

We provide some examples below, but for more information, we refer you to https://docs.python.org/2/tutorial/classes.html

In [ ]:
class myClass:
    #Class attributes are variables that are shared by all objects of this class
    teacher='Varis Carey'
    off_hours='MW 1-2 TR 12:30-1:30'
    #instance variables are variables particular to an instance of the class and are
    #set in the CONSTRUCTOR
    def __init__(self,name,room):
        self.name=name
        self.room=room
        self.students=[] #empty student list
    
    #We can have other function within myclass-here is a METHOD to add_students
    def add_student(self,name):
        self.students.append(name)

c=myClass('M5660','AB4017')
d=myClass('M4650','AB4125')
print c.name
c.add_student('Michael')
c.add_student('Megan')
print c.students
print d.students   

Let's look at an example where we encounter another example of classical object oriented programming, **Inheritance** (see https://docs.python.org/2/tutorial/classes.html#inheritance).

We start off by defining a class that solves the heat equation (a parabolic equation) that we solved numerically in Lecture II.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import axes3d
class Parabolic_Solver:
    
    #class constructor
    def __init__(self,rhs,thermal,initial,bflux,timestep=.1,delta_x=.1,delta_y=.1,time_0=0.0,time_adaptive=False,):
        self.rhs=rhs
        self.initial=initial
        self.bflux=bflux
        self.time_adaptive=time_adaptive
        self.timestep=timestep
        self.delta_x=delta_x
        self.delta_y=delta_y
        gridx=np.linspace(0,1,np.int(1.0/delta_x+1))
        gridy=np.linspace(0,1,np.int(1.0/delta_y+1))
        self.gridx,self.gridy=np.meshgrid(gridx,gridy)
        self.time=time_0
        self.init_cond()
        self.history=self.Told
        
    #definite initial temperature field.    
    def init_cond(self):
        self.Told=self.initial(self.gridx,self.gridy)

In Lecture II we used (forward) Euler to evolve a solution to the heat equation.  What happens if you make the timestep too large? You may want to go back to Lecture II and ***decrease*** the number of timesteps `N` which results in larger timesteps. You will start to see numerical solutions that become unstable. This is a well known problem with explicit schemes like the Euler method that can require very restrictive timestep sizes in order to ensure stability.
In general, we would prefer that accuracy requirements, not stability restrictions, guided our choice of timesteps.

One *cure* for this timestep restriction is to use an implicit method. The simplest such method is (backward) Euler. 

To make the notation simpler, we realize that by assigning some ordering to the countable (in fact finite) indices in the spatial mesh used for the finite difference method in space, we can first discretize in space to obtain a first-order ODE (this is a method of lines discretization). 
In other words, we have the semi-discrete version of the PDE represented as
$$
\large \frac{dT}{dt} = D T + b.
$$
Here, $T$ denotes the (vector) of temperature values at time $t$, $b$ denotes the (vector) of external source values at time $t$, and $D\in\mathbb{R}^{(M_x+1)(M_y+1)\times(M_x+1)(M_y+1)}$ is the (constant) diffusion matrix defined by the finite difference scheme in space. The diffusion matrix $D$ is also known as the stiffness matrix.

With this notation, we see that the forward Euler method simply implies that
$$
\large T^{n+1} = T^n + \Delta_t \left( D T^n + b^n \right)
$$
where $T^n$ denotes the (vector) of temperature values at time $t_n$ and $b^n$ denotes the (vector) of external source values at time $t_n$.

On the other hand, the backward Euler method evaluates the right hand side at time $t_{n+1}$, so 
$$
\large T^{n+1}= T^n + \Delta_t \left(DT^{n+1}+ b^{n+1}\right)
$$
which we may rewrite as a linear algebraic system for obtaining the new temperature,
$$
\large (I-\Delta_t D)T^{n+1} = T^n + \Delta_t b^{n+1}.
$$
The most straightforward way to solve this system **implicitly** is to assemble the stiffness matrix.  As the coefficients in the PDE are not time-dependant, we only have to do this once.

We are now going to define a **derived class** that implements the implicit solver.  

As an optional exercise, you can also add a derived class to implement the explicit Euler method we used in Lecture II.

In [ ]:
class Implicit_Time_Solver(Parabolic_Solver):  #this is a derived class from Parabolic Solver

    def __init__(self,rhs,thermal,initial,bflux,timestep=.1,delta_x=.1,delta_y=.1,time_0=0.0,time_adaptive=False): 
        # Call base class initializer
        Parabolic_Solver.__init__(self,rhs,thermal,initial,bflux,timestep,delta_x,delta_y,time_0,time_adaptive)
        # assemble the stiffness matrix
        self.assemble(thermal)
    
    def assemble(self,thermal):
        # Create an index map of the degrees of freedom 
        # that makes it easier to map spatial grid points to matrix entries
        DOF_ind=np.zeros(np.shape(self.gridx),dtype=int)
        MX=np.int(1.0/self.delta_x)+1
        MY=np.int(1.0/self.delta_y)+1
        temp=0
        for j in range(MY):    
            for i in range(MX):
                DOF_ind[i,j]=temp
                temp+=1
        self.DOF_ind=DOF_ind
        
        # Use the index map to construct the stiffness (diffusion) matrix 
        self.stiffness=np.zeros((temp,temp))
        #internal degrees of freedom
        for i in range(1,MX-1):
            
            for j in range(1,MY-1):
                
                center=DOF_ind[i,j]
                left=DOF_ind[i-1,j]
                right=DOF_ind[i+1,j]
                up=DOF_ind[i,j+1]
                down=DOF_ind[i,j-1]
                self.stiffness[center,center]=-2.0*(thermal[0]/self.delta_x**2+thermal[1]/self.delta_y**2)
                self.stiffness[center,left]=thermal[0]/self.delta_x**2
                self.stiffness[center,right]=self.stiffness[center,left]
                self.stiffness[center,down]=thermal[1]/self.delta_y**2
                self.stiffness[center,up]=self.stiffness[center,down]
        
        #horizontal boundaries (y=0,y=1)
        for i  in range(1,MX-1):
            
            center=DOF_ind[i,0]
            left=DOF_ind[i-1,0]
            right=DOF_ind[i+1,0]
            up=DOF_ind[i,1]
            self.stiffness[center,center]=-2.0*(thermal[0]/self.delta_x**2+thermal[1]/self.delta_y**2)
            self.stiffness[center,left]=thermal[0]/self.delta_x**2
            self.stiffness[center,right]=self.stiffness[center,left]
            self.stiffness[center,up] = 2.0*thermal[1]/self.delta_y**2
            
            center=DOF_ind[i,MY-1]
            left=DOF_ind[i-1,MY-1]
            right=DOF_ind[i+1,MY-1]
            down=DOF_ind[i,MY-2]
            self.stiffness[center,center]=-2.0*(thermal[0]/self.delta_x**2+thermal[1]/self.delta_y**2)
            self.stiffness[center,left]=thermal[0]/self.delta_x**2
            self.stiffness[center,right]=self.stiffness[center,left]
            self.stiffness[center,down] = 2.0*thermal[1]/self.delta_y**2
            
        #vertical boundaries (x=0, x=1)
        for j in range(1,MY-1):
            
            center=DOF_ind[0,j]
            right=DOF_ind[1,j]
            up=DOF_ind[0,j+1]
            down=DOF_ind[0,j-1]
            self.stiffness[center,center]=-2.0*(thermal[0]/self.delta_x**2+thermal[1]/self.delta_y**2)
            self.stiffness[center,right]=2.0*thermal[0]/self.delta_x**2
            self.stiffness[center,down]=thermal[1]/self.delta_y**2
            self.stiffness[center,up]=self.stiffness[center,down]
            
            center=DOF_ind[MX-1,j]
            left=DOF_ind[MX-2,j]
            up=DOF_ind[MX-1,j+1]
            down=DOF_ind[MX-1,j-1]
            self.stiffness[center,center]=-2.0*(thermal[0]/self.delta_x**2+thermal[1]/self.delta_y**2)
            self.stiffness[center,left]= 2.0*thermal[0]/self.delta_x**2
            self.stiffness[center,down]=thermal[1]/self.delta_y**2
            self.stiffness[center,up]=self.stiffness[center,down]
        
        #Now the corners
        i = 0
        j = 0
        center=DOF_ind[i,j]
        right=DOF_ind[i+1,j]
        up=DOF_ind[i,j+1]
        self.stiffness[center,center]=-2.0*(thermal[0]/self.delta_x**2+thermal[1]/self.delta_y**2)
        self.stiffness[center,right]= 2.0*thermal[0]/self.delta_x**2
        self.stiffness[center,up]=2.0*thermal[1]/self.delta_y**2
            
        i = 0
        j = MY-1
        center=DOF_ind[i,j]
        right=DOF_ind[i+1,j]
        down=DOF_ind[i,j-1]
        self.stiffness[center,center]=-2.0*(thermal[0]/self.delta_x**2+thermal[1]/self.delta_y**2)
        self.stiffness[center,right]= 2.0*thermal[0]/self.delta_x**2
        self.stiffness[center,down]=2.0*thermal[1]/self.delta_y**2
        
        i = MX-1
        j = 0
        center=DOF_ind[i,j]
        left=DOF_ind[i-1,j]
        up=DOF_ind[i,j+1]
        self.stiffness[center,center]=-2.0*(thermal[0]/self.delta_x**2+thermal[1]/self.delta_y**2)
        self.stiffness[center,left]= 2.0*thermal[0]/self.delta_x**2
        self.stiffness[center,up]=2.0*thermal[1]/self.delta_y**2
        
        i = MX-1
        j = MY-1
        center=DOF_ind[i,j]
        left=DOF_ind[i-1,j]
        down=DOF_ind[i,j-1]
        self.stiffness[center,center]=-2.0*(thermal[0]/self.delta_x**2+thermal[1]/self.delta_y**2)
        self.stiffness[center,left]= 2.0*thermal[0]/self.delta_x**2
        self.stiffness[center,down]=2.0*thermal[1]/self.delta_y**2
        
    def advance(self):
        MX=np.int(1.0/self.delta_x)+1
        MY=np.int(1.0/self.delta_y)+1
        Mat=np.eye(MX*MY)-self.timestep*self.stiffness
        data=self.Told  #will overwrite boundary terms.
        #compute right hand side
        #internal degrees of freedom
        #evaluate rhs, including boundary
        temp=self.timestep*self.rhs(self.gridx,self.gridy,self.time)
        temp=np.reshape(temp,np.size(data))
        #interal DOF
        data[self.DOF_ind[1:MX-2,1:MY-2]]+=temp[self.DOF_ind[1:MX-2,1:MY-2]]
        #boundary conditions
        for i in range(0,MX):
            data[self.DOF_ind[i,0]] += self.timestep*2.0*thermal[1] / \
                                        self.delta_y * self.bflux(self.gridx[0,i],0.0,self.time)
            data[self.DOF_ind[i,MY-1]] += self.timestep*2.0*thermal[1] / \
                                        self.delta_y * self.bflux(self.gridx[MY-1,i],1.0,self.time)
        for j in range(0,MY):
            data[self.DOF_ind[0,j]] += self.timestep*2.0*thermal[0] / \
                                        self.delta_x * self.bflux(0.0,self.gridy[j,0],self.time)
            data[self.DOF_ind[MX-1,j]] += self.timestep*2.0*thermal[0] / \
                                        self.delta_x * self.bflux(1.0,self.gridy[j,MX-1],self.time)
        self.Tnew=np.linalg.solve(Mat,data)
        self.time+=self.timestep
        self.Told=self.Tnew
        self.history=np.vstack((self.history,self.Told))
    
            
    # We also need to redefine the initial condition function to work with our solution vector.
    def init_cond(self):
        Parabolic_Solver.init_cond(self)
        #make the solution vector one dimensional for the linear algebra.
        self.Told=np.reshape(self.Told,np.size(self.Told))  
             
    # plotter for a specific timestep.
    def plot_state(self,which_step):
        # construct meshgrid vector from history vector for plotting
        MX,MY=np.shape(self.gridx)
        plotwhat=np.zeros((MX,MY))
        for i in range(MX):
            for j in range(MY):
                plotwhat[i,j]=self.history[which_step+1,self.DOF_ind[i,j]]
        #print plotwhat
        fig = plt.figure()
        ax1 = fig.add_subplot(1, 1, 1, projection='3d')

        ax1.plot_wireframe(self.gridx, self.gridy, plotwhat) 
        ax1.set_xlabel('x')
        ax1.set_ylabel('y')
        ax1.set_zlabel('T')
        titstring='Approx. Solution at Timestep '+str(which_step+1)
        ax1.set_title(titstring)        

    def plot_error(self, which_step, T_exact):
        MX,MY=np.shape(self.gridx)
        plotwhat=np.zeros((MX,MY))
        for i in range(MX):
            for j in range(MY):
                plotwhat[i,j]=self.history[which_step+1,self.DOF_ind[i,j]]
        #print plotwhat
        fig = plt.figure()
        ax1 = fig.add_subplot(1, 1, 1, projection='3d')
        
        ax1.plot_wireframe(self.gridx, self.gridy, T_exact-plotwhat, color='red')
        ax1.set_xlabel('x')
        ax1.set_ylabel('y')
        ax1.set_zlabel('error')
        titstring='Error in Solution at Timestep '+str(which_step+1)
        ax1.set_title(titstring)    

This code block corresponds to running our actual problem from Lecture II but using the backward Euler time integrator.

In [ ]:
def rhs(x_i, y_j, t_n):
    return np.zeros(np.shape(x_i))

def initial(x,y):
    ic = 10*np.sin(np.pi*x)*np.sin(np.pi*y)
    return ic

def bflux(x_i, y_j, t_n):
    if x_i == 0.0 or x_i == 1.0:
        z = -10*np.pi*np.exp(-t_n)*np.sin(np.pi*y_j)
    else:
        z = -10*np.pi*np.exp(-t_n)*np.sin(np.pi*x_i)
    return z

thermal=np.array([1/(2*np.pi**2),1/(2*np.pi**2)])

def T_exact(x,y,t):
    return 10*np.exp(-t)*np.sin(np.pi*x)*np.sin(np.pi*y)

delta_t = 0.2
delta_x = 0.1
delta_y = 0.1
test=Implicit_Time_Solver(rhs,thermal,initial,bflux,delta_t,delta_x,delta_y,0,False)
for i in range(10):
    test.advance()
    test.plot_state(i)
    test.plot_error(i, T_exact(test.gridx, test.gridy, test.timestep*(i+1)))

In [ ]:
#Optional Exercise
class Explicit_Time_Solver(Parabolic_Solver):
        
    def __init__(self,):
        
    def advance()
    